## 환경 설정 및 모듈 임포트

In [ ]:
# ==========================================================
# 1. 라이브러리 및 사용자 정의 모듈 불러오기
# ==========================================================
import pandas as pd
import sys
import os

# ----------------------------------------------------------
# 현재 노트북 경로를 시스템 경로에 추가하여 
# scripts 폴더 내 모듈(data_loader.py 등)을 불러올 수 있도록 설정
# ----------------------------------------------------------
sys.path.append('../scripts')

# ----------------------------------------------------------
# data_loader 모듈 임포트 및 BASE_PATH 확인
# ----------------------------------------------------------
import data_loader 
print(f"data_loader.py의 BASE_PATH 확인: {data_loader.BASE_PATH}")

print("--- 모듈 임포트 및 환경 설정 완료 ---")

## 데이터 로딩, 클리닝 및 병합

In [ ]:
# ==========================================================
# 1. 데이터 로딩
# ==========================================================
data_dict = data_loader.load_all_data()

# ==========================================================
# 2. 초기 클리닝 및 날짜 타입 변환
# ==========================================================
data_dict = data_loader.clean_initial_data(data_dict)

# ==========================================================
# 3. 핵심 4개 테이블 병합 실행
# ==========================================================
df_merged = data_loader.merge_core_tables(data_dict)

# ==========================================================
# 4. 병합 결과 확인
# ==========================================================
print("\n--- 통합 데이터셋 (df_merged) 구조 확인 ---")
print(f"총 행 수: {df_merged.shape[0]:,} | 총 컬럼 수: {df_merged.shape[1]}")
display(df_merged.head())

## feature_engineer 모듈 임포트 및 특성 생성

In [ ]:
# ==========================================================
# Cell 3: feature_engineer 모듈 임포트 및 RFM 생성
# ==========================================================

import feature_engineer as fe

print("--- feature_engineer 모듈 임포트 완료 ---")

# ==========================================================
# 1. 고객 특성 생성
# ==========================================================
df_customer_features = fe.create_customer_features(df_merged)

# ==========================================================
# 2. 타겟 변수 정의 (이탈 기준 365일)
# ==========================================================
# Recency를 기반으로 'Target_Churn' 변수를 생성.
df_final_ml = fe.define_churn_target(df_customer_features, churn_period_days=365)

# ==========================================================
# 3. 최종 ML 입력 데이터셋 확인
# ==========================================================
print("--- 최종 ML 입력 데이터셋 확인 ---")
print(f"고객 수: {df_final_ml.shape[0]:,}명 | 컬럼 수: {df_final_ml.shape[1]}개")

## R_Recency 분포 시각화 및 이탈 기준 검토

In [ ]:
# ==========================================================
# Cell 4: R_Recency 분포 시각화 및 이탈 기준 검토 (365일 적용)
# ==========================================================

import matplotlib.pyplot as plt
import seaborn as sns
import platform 
from matplotlib import font_manager, rc 

# ==========================================================
# 1. 폰트 설정 (한글 깨짐 방지)
# ==========================================================
if platform.system() == 'Darwin':  # Mac OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # Windows OS
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic') 

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 폰트 깨짐 방지

print("--- R_Recency 분포 시각화 시작 (이탈 기준 365일 검증) ---")

# ==========================================================
# 2. R_Recency (최신성) 분포 시각화
# ==========================================================
plt.figure(figsize=(10, 6))
sns.histplot(df_final_ml['R_Recency'], bins=50, kde=True, binrange=(0, 750))

# ==========================================================
# 3. 새로운 이탈 기준선(365일) 표시
# ==========================================================
current_churn_days = 365
plt.axvline(x=current_churn_days, color='b', linestyle='-', label=f'새 이탈 기준 ({current_churn_days}일)')

# ==========================================================
# 4. 그래프 스타일 설정 및 출력
# ==========================================================
plt.title('R_Recency (최근 구매일로부터 경과 일수) 분포')
plt.xlabel('경과 일수 (일)')
plt.ylabel('고객 수')
plt.legend()
plt.grid(axis='y', alpha=0.5)
plt.show()

print("\n--- 시각화 결과 분석: 365일 기준으로 데이터가 유지/이탈로 잘 나뉘는지 확인 ---")

## F, M 지표 분포 시각화 (로그 변환 필요성 검토)

In [ ]:
# ==========================================================
# Cell 5: F_Frequency, M_Monetary 분포 시각화 및 이상치 검토
# ==========================================================

print("--- F, M 지표 분포 시각화 시작 ---")

plt.figure(figsize=(15, 5))

# ==========================================================
# 1. Frequency (F) 분포
# ==========================================================
# Frequency가 0이 아닌 값만 시각화. 
# (대부분의 Frequency는 1이므로 1 주변의 분포 확인)
# df_final_ml['F_Frequency']는 이미 정수형이므로, 1보다 큰 값을 확인.
plt.subplot(1, 2, 1)
sns.histplot(df_final_ml.loc[df_final_ml['F_Frequency'] >= 1, 'F_Frequency'], bins=30, kde=False)
plt.title('F_Frequency (주문 빈도) 분포')
plt.xlabel('주문 건수')
plt.ylabel('고객 수')

# ==========================================================
# 2. Monetary (M) 분포
# ==========================================================
# Monetary 값이 큰 이상치가 있을 가능성이 높으므로 상위 99% 값까지만 시각화.
plt.subplot(1, 2, 2)
max_monetary = df_final_ml['M_Monetary'].quantile(0.99)
sns.histplot(df_final_ml.loc[df_final_ml['M_Monetary'] < max_monetary, 'M_Monetary'], bins=30, kde=False)
plt.title('M_Monetary (총 지출 금액) 분포 (상위 1% 제외)')
plt.xlabel('총 지출 금액')
plt.ylabel('고객 수')

# ==========================================================
# 3. 그래프 출력
# ==========================================================
plt.tight_layout()
plt.show()

print("\n--- 시각화 결과 분석 ---")

## RFM 지표 로그 변환 및 데이터 저장

In [ ]:
# ==========================================================
# Cell 6: F, M, 배송 시간 로그 변환, 인코딩 및 최종 데이터 저장
# ==========================================================

import numpy as np 
import os 
import pandas as pd 

print("--- 최종 데이터 전처리 시작 (로그 변환 & 인코딩) ---")

# ==========================================================
# 1. F, M, 배송 시간 특성 로그 변환
# ==========================================================
# Recency 관련 특성과 상호작용 특성은 이미 제거했거나 사용하지 않음
df_final_ml['F_Frequency_log'] = np.log1p(df_final_ml['F_Frequency'])
df_final_ml['M_Monetary_log'] = np.log1p(df_final_ml['M_Monetary'])

# Avg_Delivery_Time은 연속형 특성이므로 로그 변환 적용
# 결측치(NaN)는 0으로 처리되어 있어 log1p 적용 가능
df_final_ml['Avg_Delivery_Time_log'] = np.log1p(df_final_ml['Avg_Delivery_Time'])

# ==========================================================
# 2. 범주형 변수 인코딩 (One-Hot Encoding)
# ==========================================================
# 'customer_state'만 인코딩 (payment_type은 이미 OHE 후 합산)
df_encoded = pd.get_dummies(df_final_ml, columns=['customer_state'], drop_first=True, prefix='state') 

# ==========================================================
# 3. 불필요한 원본 컬럼 제거
# ==========================================================
# 원본 RFM 값, 원본 배송 시간, customer_unique_id 제거
df_encoded = df_encoded.drop(columns=[
    'customer_unique_id', 
    'R_Recency', 'F_Frequency', 'M_Monetary', 
    'Avg_Delivery_Time'
])

df_final_ml = df_encoded.copy()

print("--- 로그 변환 및 인코딩 완료 ---")
print(f"새로운 컬럼 수: {df_final_ml.shape[1]}개")
print(f"인코딩된 주(State) 컬럼 예시: {', '.join([col for col in df_final_ml.columns if col.startswith('state_')][:3])}...")

# ==========================================================
# 4. 최종 데이터셋 저장
# ==========================================================
OUTPUT_FILE = '../data/processed/df_final_ml.csv'
os.makedirs('../data/processed', exist_ok=True) 
df_final_ml.to_csv(OUTPUT_FILE, index=False)

print("--- 최종 데이터셋 저장 완료 ---")
print(f"저장 경로: {OUTPUT_FILE}")